# Frozen Lake 강화학습 연습

<img src="./img/frozenlake_env.png" width="700">

## 필요한 모듈 import
* gym (가상환경을 제공) 
* numpy (텐서 연산)
* matplotlib (결과 그래프 출력)

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt 

## 가상환경 설정 및 hyper parameter 정의
* gym.make를 이용하여 등록된 'FrozenLake-v0' 환경을 env로 정의, is_slippery=True $\rightarrow$ Transition Prob = 1.0
* learning rate (학습률)
* dis (감쇄계수)
* num_episodes(전체 반복 학습)
* env.render(): 설정한 가상환경을 보고싶을 때, S: Start, F: Frozen, H: Hole, G: Goal, 붉은 박스: 현재 위치

In [ ]:
env = gym.make('FrozenLake-v0', is_slippery=False)
learning_rate = 0.8
dis = .95
num_episodes = 2000

# env.render()

## 펭수가 학습할 가상환경 관찰
* info = 상태전이 확률 (if slippery = true, then prob = 0.333333)
* env.render(): 설정한 가상환경을 보고싶을 때, S: Start, F: Frozen, H: Hole, G: Goal, 붉은 박스: 현재 위치

In [ ]:
env.reset()
for i in range(10):
    random_action = env.action_space.sample()
    new_state, reward, done, info = env.step(random_action)
    env.render()
    print(info)
    if done:
      break

## Q-Table 및 보상값 리스트 정의
* Q = Q-Table [num_states, num_actions]

In [ ]:
Q = np.zeros([env.observation_space.n, env.action_space.n])
rList = []

print("number of states: ", env.observation_space.n)
print("number of actions: ", env.action_space.n)

## 학습 시작
* num_episodes 만큼 반복
* 한번 에피소드 당 99번 반복하거나 terminal state (Hole, Goal)에 도착하면 다음 에피소드로
*  $\epsilon$ - greedy를 적용하여  $\epsilon$ = 1/(n+1) 확률 만큼 maxQ(s,a)가 아닌 액션을 취함 (이때 n은 100번 에피소스마다 1씩 증가)

In [ ]:
for i in range(num_episodes):
    
    # env 리셋
    state = env.reset()
    rAll = 0
    done = False
    
    e = 1./((i//100)+1) # decaying E-greedy
    
    j = 0
    
    # 99번 반복하거나 terminal state (Hole, Goal) 도착
    while j < 99:
        
        j += 1
        
        # egreedy에 의한 행동 설정
        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q[state, :])

        # new_state, reward 업데이트 
        new_state, reward, done, _ = env.step(action)
        
        # 업데이트 Q-table (상태, 행동)
        Q[state, action] = Q[state, action] + learning_rate*( reward + dis*np.max(Q[new_state, :]) - Q[state, action] )
        
        rAll += reward
        state = new_state

        if done == True:
            break
        
    rList.append(rAll)

In [ ]:
print('성공율: ', str(sum(rList)/num_episodes))
print('Q-table')
print(Q)
plt.bar(range(len(rList)), rList, color = 'blue')
plt.show()